In [73]:
import django, sys, os
import numpy as np
import pandas as pd

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()
from django.db.models import Q, F, Sum, Count, FloatField, Case, When, Value, Max
from scipy.sparse import coo_matrix, find
from sklearn.manifold import TSNE

from scoping.models import *
from tmv_app.models import *

run_id = 665

In [71]:
stat = RunStats.objects.get(pk=run_id)
if stat.method=="DT":
    topics = DynamicTopic
    doctopics = DocDynamicTopic
else:
    topics = Topic
    doctopics = DocTopic
    
top_topics = list(topics.objects.filter(run_id=run_id).order_by('-share').values())

for t in top_topics:
    t['top words'] = t['top_words']
    t['top_docs'] = doctopics.objects.filter(topic_id=t['id']).order_by('-score').values_list('doc__title',flat=True)[:2]
    split_docs = "; \\\\".join(t['top_docs'])
    t['top docs'] = f"\parbox[t]{{7.5cm}}{{{split_docs}}}"
    t['title'] = t['title'].replace('{','').replace('}','')
    t['share'] = str(round(t['share']*100,2)) + '\%'

df = pd.DataFrame.from_dict(top_topics)

df = df[['title','top words','top docs','share']]
#df['share'] = format(df['share'],".0%")
with pd.option_context("max_colwidth", 1000):
    df.head(10).to_latex(
        '../tables/top_topics.tex',
        column_format="p{1.9cm} p{3cm} p{7.5cm} r", 
        #float_format='{:.2%}'.format,
        index=False,
        escape=False
    )

df.head()
    


,title,top words,top docs,share
0,"climat, chang, impact","[climat, chang, impact, respons, futur, effect...",\parbox[t]{7.5cm}{Climate oscillations and cha...,2.73\%
1,"soil, moistur, microbi","[soil, moistur, microbi, organ, respir, conten...",\parbox[t]{7.5cm}{PARTITIONING OF SOIL RESPIRA...,2.73\%
2,"emiss, reduct, reduc","[emiss, reduct, reduc, greenhous, factor, tota...",\parbox[t]{7.5cm}{China's CH4 and CO2 emission...,2.21\%
3,"carbon, dioxid, sequestr","[carbon, dioxid, sequestr, sink, organ, cycl, ...",\parbox[t]{7.5cm}{Interpreting carbon-isotope ...,1.74\%
4,"temperatur, air, mean","[temperatur, air, mean, surfac, minimum, maxim...",\parbox[t]{7.5cm}{Observed changes in shallow ...,1.71\%
